In [ ]:
import os
import sys
import json
import urllib
import zipfile
from collections import defaultdict
from datetime import datetime
import tempfile

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from azureml.core import Workspace, Model, Dataset

from tqdm.notebook import tqdm

from IPython.display import display

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from P10_03_luis.utils import *

In [ ]:
# On définit les variables globales
JSON_PATH = "data/json/"
MODELS_PATH = "data/models/"

os.makedirs(JSON_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

RANDOM_SEED = 42

In [ ]:
def pprint_dict(data: dict):
    """Pretty print a dict"""
    print(json.dumps(data, indent=2))

# Introduction

## Le jeu de données

In [ ]:
# Lien vers le dataset
DATASET_URL = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+10%C2%A0-+D%C3%A9veloppez+un+chatbot+pour+r%C3%A9server+des+vacances/frames.zip"
DATASET_FN = "frames.json"

# On vérifie si le fichier est bien présent
if DATASET_FN in os.listdir(JSON_PATH):
    print("Tous les fichiers sont bien présents.")
# Sinon on télécharge et on extrait le fichier
else:
    print("Téléchargement des données en cours...")

    # On télécharge le .zip dans un fichier temporaire et on extrait les données
    tmp, _ = urllib.request.urlretrieve(DATASET_URL)
    with zipfile.ZipFile(tmp, "r") as f:
        # On extrait le fichier json
        f.extract(DATASET_FN, JSON_PATH)

    # On supprime le fichier temporaire
    urllib.request.urlcleanup()
    
    print("Téléchargement des données terminé.")

# Chargement des ressources

## Chargement du workspace

In [ ]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin des données

In [ ]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

# Exploration et analyse des données

## Chargement du fichier

In [ ]:
# On charge les frames
with open("data/json/frames.json") as f:
    frames = json.load(f)

In [ ]:
frames[0]

In [ ]:
list(frames[0].keys())

In [ ]:
list(frames[0]["turns"][0].keys())

## Analyse des textes

In [ ]:
# On affiche le premier dialogue
for turn in frames[0]["turns"]:
    print("-" * 80)
    print(turn.get("text", ""))

In [ ]:
# On affiche les premières utterances des 10 premiers dialogues
for i in range(10):
    print(frames[i].get("turns", [])[0].get("text", ""))

## Analyse des labels

In [ ]:
pprint_dict(frames[0]["turns"][0]["labels"]["acts_without_refs"])

In [ ]:
keys = defaultdict(set)
for i in frames:
    for j in i["turns"][0]["labels"]["acts_without_refs"]:
        for k in j["args"]:
            try:
                keys[k["key"]].add(k.get("val", ""))
            except:
                print(k)

In [ ]:
{k: len(v) for k, v in keys.items()}

In [ ]:
keys["intent"]

## Transformation des données

In [ ]:
label_to_entity = {
    "or_city": "from_city",
    "dst_city": "to_city",
    "str_date": "from_dt",
    "end_date": "to_dt",
    "budget": "budget"
}

intent_name = "book_flight"

def turn_to_luis_utterance(turn, intent_name, label_to_entity):
    """"""
    text = turn["text"]

    entity_labels = []
    for i in turn["labels"]["acts_without_refs"]:
        for l in i["args"]:
            k = l["key"]
            v = l["val"]

            if k and v:
                if k in label_to_entity.keys():
                    start_char_index = text.lower().find(v.lower())
                    if start_char_index == -1:
                        continue
                    
                    end_char_index = start_char_index + len(v) - 1

                    entity_labels.append({
                        "startCharIndex": start_char_index,
                        "endCharIndex": end_char_index,
                        "entityName": label_to_entity[k],
                    })
    
    res = {
        "text": text,
        "intentName": intent_name,
        "entityLabels": entity_labels,
    }
    return res

res = turn_to_luis_utterance(frames[0]["turns"][0], intent_name, label_to_entity)
pprint_dict(res)

In [ ]:
res = []
for frame in frames:
    user_turn_id = 0
    for turn in frame["turns"]:
        if turn["author"] == "user":
            row = {"user_turn_id": user_turn_id}
            user_turn_id += 1
            
            row.update(turn_to_luis_utterance(turn, intent_name, label_to_entity))
        
            res.append(row)

In [ ]:
df = pd.DataFrame(res)
df["entity_total_nb"] = df["entityLabels"].apply(len)

for entity_name in label_to_entity.values():
    df[f"{entity_name}_nb"] = df["entityLabels"].apply(
        lambda x: len(list(
            filter(lambda x1: x1["entityName"] == entity_name, x)
        ))
    )

df["text_word_nb"] = df["text"].apply(lambda x: len(x.split()))

none_intent_mask = df["entity_total_nb"] == 0
df.loc[none_intent_mask, "intentName"] = "None"
    
df.shape

In [ ]:
df.head()

In [ ]:
df[
    ["text_word_nb", "entity_total_nb"] + [f"{i}_nb" for i in label_to_entity.values()]
].describe().T

In [ ]:
book_flight_utterances = df[(df["user_turn_id"] == 0) & (df["entity_total_nb"] == len(label_to_entity) - 1)]
len(book_flight_utterances)

In [ ]:
none_utterances = df[(df["user_turn_id"] == 0) & (df["entity_total_nb"] == 0)]
len(none_utterances)

In [ ]:
none_utterances = none_utterances.sample(len(book_flight_utterances), random_state=RANDOM_SEED)
len(none_utterances)

In [ ]:
utterances = pd.concat([book_flight_utterances, none_utterances])

## Split des données

In [ ]:
utterances_train = utterances.sample(frac=0.7, random_state=RANDOM_SEED)

In [ ]:
utterances_test_idx = utterances.index.difference(utterances_train.index)
utterances_test = utterances.loc[utterances_test_idx]

In [ ]:
utterances_train.shape, utterances_test.shape

In [ ]:
utterances_train = utterances_train[["text", "intentName", "entityLabels"]].to_dict("records")

In [ ]:
pprint_dict(utterances_train[0])

In [ ]:
utterances_test = utterances_test[["text", "intentName", "entityLabels"]].to_dict("records")

In [ ]:
from typing import List, Dict

from pydantic import BaseModel

In [ ]:
class EntityBaseModel(BaseModel):
    entity: str
    startPos: int
    endPos: int
    children: List["EntityBaseModel"] = []

    def __init__(self, **kwargs):
        kwargs["entity"] = kwargs["entityName"]
        kwargs["startPos"] = kwargs["startCharIndex"]
        kwargs["endPos"] = kwargs["endCharIndex"]
        super().__init__(**kwargs)

class TestUterranceBaseModel(BaseModel):
    text: str
    intent: str
    entities: List[EntityBaseModel]

    def __init__(self, **kwargs):
        kwargs["intent"] = kwargs["intentName"]
        kwargs["entities"] = kwargs["entityLabels"]
        super().__init__(**kwargs)
        
def conv_example_to_utterance_format(utterance):
    """"""
    return TestUterranceBaseModel(**utterance).dict()

In [ ]:
utterances_test = {
    "LabeledTestSetUtterances": list(map(conv_example_to_utterance_format, utterances_test))
}

In [ ]:
utterances_test["LabeledTestSetUtterances"][0]

## Enregistrement des datasets

In [ ]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    # On enregistre les données
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path, "w") as f:
        json.dump(list(map(conv_example_to_utterance_format, utterances_train)), f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path, "w") as f:
        json.dump(utterances_test, f)
    
    # On upload tous les fichiers dans le datastore
    ds = Dataset.File.upload_directory(
        tmp_dir_name,
        target=(datastore, "utterances/" + datetime.now().strftime("%Y_%m_%d")),
        overwrite=True,
        show_progress=True
    )

In [ ]:
ds = ds.register(
    workspace=ws,
    name="utterances",
    description="Train and test utterances",
    create_new_version=True
)

In [ ]:
ds.version

## Chargement des données

In [ ]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    dataset = Dataset.get_by_name(ws, name='utterances')
    dataset.download(target_path=tmp_dir_name, overwrite=False)
    
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path) as f:
        train_json = json.load(f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path) as f:
        test_json = json.load(f)

In [ ]:
train_json[0]

In [ ]:
test_json["LabeledTestSetUtterances"][0]

# Création du modèle LUIS

In [ ]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject, AzureAccountInfoObject, LuisApp
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

import json, time, uuid

In [ ]:
from dotenv import load_dotenv, set_key

load_dotenv("../P10_03_luis/.env")

In [ ]:
LUIS_AUTH_KEY = os.getenv("LUIS_AUTH_KEY")
LUIS_AUTH_ENDPOINT = os.getenv("LUIS_AUTH_ENDPOINT")

LUIS_PRED_KEY = os.getenv("LUIS_PRED_KEY")
LUIS_PRED_ENDPOINT = os.getenv("LUIS_PRED_ENDPOINT")

In [ ]:
# On s'authentifie
client = LUISAuthoringClient(LUIS_AUTH_ENDPOINT, CognitiveServicesCredentials(LUIS_AUTH_KEY))

In [ ]:
# We use a UUID to avoid name collisions.
app_name = "p10-luis-app"
app_version = "0.1"
intent_name = "book_flight"

In [ ]:
# define app basics
app_def = ApplicationCreateObject(name=app_name, initial_version_id=app_version, culture='en-us')

# create app
app_id = client.apps.add(app_def)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

In [ ]:
set_key("../P10_03_luis/.env", "LUIS_APP_ID", app_id)

In [ ]:
client.model.add_intent(app_id, app_version, intent_name)

In [ ]:
# Add Prebuilt entity
client.model.add_prebuilt(
    app_id,
    app_version,
    prebuilt_extractor_names=[
        "number",
        "datetimeV2",
        "geographyV2"
    ]
)

In [ ]:
entity_to_id = {
    "from_city": "",
    "to_city": "",
    "from_dt": "",
    "to_dt": "",
    "budget": ""
}

In [ ]:
for entity_name in entity_to_id.keys():
    # add entity to app
    entity_to_id[entity_name] = client.model.add_entity(
        app_id,
        app_version,
        name=entity_name
    )

entity_to_id

In [ ]:
# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "from_phrase_list",
        "phrases": "from,start at,begin from,leave"
    }
)

# add phrase list to app
client.features.add_phrase_list(
    app_id,
    app_version,
    {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "to_phrase_list",
        "phrases": "to,arrive,land at,go,going,stay,heading"
    }
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["to_city"],
    {"feature_name": "to_phrase_list", "model_name": None}
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["from_city"],
    {"model_name": "geographyV2", "is_required": True}
)

# add phrase list as feature to subentity model
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["from_city"],
    {"feature_name": "from_phrase_list", "model_name": None}
)

In [ ]:
client.features.add_entity_feature(
    app_id,
    app_version,
    entity_to_id["budget"],
    {"model_name": "number", "is_required": True}
)

In [ ]:
train_json

In [ ]:
# for utterance in train_json:
#     try:
#         client.examples.add(app_id, app_version, utterance, { "enableNestedChildren": True })
#     except:
#         print(json.dumps(utterance, indent=2))
        
client.examples.batch(
    app_id,
    app_version,
    train_utterances,
    enable_nested_children=True
)

In [ ]:
train(app_version)

In [ ]:
deploy(app_version, is_staging=True)

In [ ]:
pred = get_prediction(
    is_staging=True,
    utterance="I want to go from Paris to London the 25/12/2021 and stay one week. I have only 512€."
)

pprint_dict(pred)

In [ ]:
res = evaluate(is_staging=True, utterances=test_utterances)
res

## Enregistrement des paramètres du modèle

In [ ]:
params = get_params(app_version)

In [ ]:
params["utterances"] = []
params["versionId"] = 0.1

In [ ]:
model_config = {
    "model": params,
    "dataset": {
        "name": ds.name,
        "version": ds.version,
    }
}

In [ ]:
file_path = os.path.join("../P10_03_luis", "params.json")
with open(file_path, "w") as f:
    json.dump(model_config, f)

## Suppression du modèle

In [ ]:
delete(app_version)

In [ ]:
azure_workspace = {
  "resourceGroup": "p10-ml-rg",
  "workspaceName": "p10-ml-ws"
}